In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

# Loads the image

In [ ]:
def read_nifti(file_name, labels = ['X', 'Y', 'Z', 'T']):
    
    img = nib.load(file_name)
    
    return img, img.shape

In [ ]:
# open image

file_name = '/home/lenkeiuser/Documents/JC/1_DATA/2023-06-14_python-course/hungary/sub-Kremes/ses-CtrlH/func/sub-Kremes_ses-CtrlH_task-stim_run-01_pd2dt.nii.gz'

img, [X,Y,Z,T] = read_nifti(file_name)

print(f'Image loaded\nX: {X}, Y: {Y}, Z: {Z}, T:{T}')

In [ ]:
# open mask

mask_name = '/home/lenkeiuser/Documents/JC/1_DATA/2023-06-14_python-course/hungary/sub-Kremes/ses-CtrlH/func/sub-Kremes_ses-CtrlH_task-stim_run-01_mask.nii.gz'

mask_img, [X,Y,Z] = read_nifti(mask_name)

print(f'Mask loaded\nX: {X}, Y: {Y}, Z: {Z}')

In [ ]:
# Checks header

for field in img.header:
    
    print(f'{field:<18} : {img.header[field]}')

In [ ]:
# Just the fields of interest

fields_of_interest = ['dim', 'pixdim', 'srow_x', 'srow_y', 'srow_z']

for field in fields_of_interest:
    
    print(f'{field:<18} : {img.header[field]}')

In [ ]:
# Check the affine

print(img.affine)

In [ ]:
# Check the shapes

print(f'image: {img.header["dim"]}\nmask: {mask_img.header["dim"]}')

In [ ]:
# Compare the affines

print(f'image: \n{img.affine}\nmask: \n{mask_img.affine}')

# Orientation

In [ ]:
# proper display

from nilearn.plotting import plot_anat
from nilearn.image import mean_img, math_img

display = plot_anat(mean_img(img))
display.add_overlay(mask_img, alpha = 0.5)


In [ ]:
# Custom display

# get images to display

img_data = img.get_fdata()
mask_data = mask_img.get_fdata()

with np.errstate(divide = 'ignore', invalid = 'ignore'):
    img_0 = np.log10(img_data.squeeze().mean(2))
img_1 = mask_data.squeeze()

In [ ]:
# Displays with matplotlib

plt.imshow(np.rot90(img_0), cmap = 'Greys_r', aspect = 0.1/0.11, vmin = 0.6)
plt.imshow(np.rot90(img_1), cmap = 'Greys_r', alpha = 0.5, aspect = 0.1/0.11)

plt.axis('off')

In [ ]:
# Is rotating the solution

plt.imshow(np.rot90(img_0,3), cmap = 'Greys_r', aspect = 0.1/0.11, vmin = 0.6)
plt.imshow(np.rot90(img_1,3), cmap = 'Greys_r', alpha = 0.5, aspect = 0.1/0.11)

plt.axis('off')

In [ ]:
# It is different from a flip

plt.imshow(np.rot90(np.flip(img_0,1)), cmap = 'Greys_r', aspect = 0.1/0.11, vmin = 0.6)
plt.imshow(np.rot90(np.flip(img_1,1)), cmap = 'Greys_r', alpha = 0.5, aspect = 0.1/0.11)

plt.axis('off')

In [ ]:
# Never mess with the affine

affine = mask_img.affine
affine[2,2] = 0.1
flipped_mask = nib.Nifti1Image(
    mask_img.get_fdata(), 
    affine = affine, 
    header = mask_img.header
)

flipped_mask.affine

In [ ]:
# Changing the affine is changing the world representation

display = plot_anat(mean_img(img))
display.add_overlay(flipped_mask, alpha = 0.5)

In [ ]:
# Modifying the affine requires geometrical processing

affine = mask_img.affine
affine[2,2] = 0.1
affine[2,3] = -Z*0.1
flipped_mask = nib.Nifti1Image(
    mask_img.get_fdata(), 
    affine = affine, 
    header = mask_img.header
)

flipped_mask.affine

In [ ]:
display = plot_anat(mean_img(img))
display.add_overlay(flipped_mask, alpha = 0.5)

In [ ]:
# Practicals 2.3.0

# Correlation analysis

In [ ]:
# open image

file_name = '/home/lenkeiuser/Documents/JC/1_DATA/2023-06-14_python-course/hungary/sub-Kremes/ses-CtrlH/func/sub-Kremes_ses-CtrlH_task-stim_run-01_pd2dt.nii.gz'
mask_name = '/home/lenkeiuser/Documents/JC/1_DATA/2023-06-14_python-course/hungary/sub-Kremes/ses-CtrlH/func/sub-Kremes_ses-CtrlH_task-stim_run-01_mask.nii.gz'

img, [X,Y,Z,T] = read_nifti(file_name)

print(f'Image loaded\nX: {X}, Y: {Y}, Z: {Z}, T:{T}')

mask_img, [X,Y,Z] = read_nifti(mask_name)

print(f'Mask loaded\nX: {X}, Y: {Y}, Z: {Z}')

In [ ]:
# Display proper

from nilearn.plotting import plot_carpet

_ = plot_carpet(img, mask_img = mask_img, t_r = 0.4)

In [ ]:
# Custom display

from nilearn.masking import apply_mask, unmask

signals = apply_mask(img, mask_img)

print(signals.shape)

In [ ]:
# Weird

plt.imshow(np.rot90(signals), interpolation = 'none', cmap = 'Greys_r', aspect = 'auto')

In [ ]:
# Normalise

signals_norm = (signals-signals.mean(0))/signals.std(0)

In [ ]:
# smooth custom

plt.imshow(
    np.rot90(signals_norm), 
    cmap = 'Greys_r', 
    aspect = 'auto', 
    vmin = -2, 
    vmax = 2
)

In [ ]:
# Proper is not always that proper

plt.imshow(
    np.rot90(signals_norm_clean), 
    cmap = 'Greys_r', 
    aspect = 'auto', 
    interpolation = 'none',
    vmin = -2, 
    vmax = 2
)

In [ ]:
# Looking at global signal

plt.plot(signals_norm.mean(1))

In [ ]:
# proper detrend

from scipy.signal import detrend

signals_detrend = detrend(signals_norm, type = 'linear', axis = 0)

In [ ]:
# not so proper

plt.plot(signals_detrend.mean(1))

In [ ]:
# Custom detrend

def detrend_poly(signals, deg = 3):
    
    [T,nb_vox] = signals.shape
    x = np.linspace(-1, 1, T)

    p = np.polyfit(x, signals, deg)
    poly_fit = np.array([np.polyval(p_i, x) for p_i in p.T]).T
    
    return signals-poly_fit


signals_detrend = detrend_poly(signals_norm)

In [ ]:
# detrended signal

plt.plot(signals_detrend.mean(1))

In [ ]:
# Lets go back to images

img_norm = unmask(signals_norm, mask_img)
img_detrend = unmask(signals_detrend, mask_img)

In [ ]:
# Custom display

def display_fus_img(img, fig = None, ax = None):
    
    if not fig:
        
        fig = plt.figure(figsize = (6,5), dpi = 200)
        
    if not ax:
        
        ax = fig.gca()        
    
    img_0 = np.flip(img.get_fdata().squeeze().mean(2), 1)
        
    ax.imshow(np.rot90(img_0), cmap = 'Greys_r')
    ax.axis('off')
    
    return fig, ax
    
# Normalised

_ = display_fus_img(img_norm)

In [ ]:
# Detrended

_ = display_fus_img(img_detrend)

In [ ]:
# Lindquist 2018

# Getting the stim

In [ ]:
# Reading events file

from pandas import read_csv

fs = 2.5

csv_name = file_name = '/home/lenkeiuser/Documents/JC/1_DATA/2023-06-14_python-course/hungary/sub-Kremes/ses-CtrlH/func/sub-Kremes_ses-CtrlH_task-stim_run-01_events.tsv'

events = read_csv(csv_name, delimiter = '\t')
events

In [ ]:
# Building event regressor

event_signal = np.zeros(T)
for row in events.iterrows():
    
    if row[1].trial_type == 'active':

        event_signal[int(row[1].onset*fs):int(row[1].onset*fs)+int(row[1].duration*fs)] = 1

In [ ]:
# plotting the stim

plt.plot(event_signal)

# Correlation map

In [ ]:
# Computes correlation

cor_raw = [np.corrcoef(x, event_signal)[0,1] for x in signals.T]
cor_detrend = [np.corrcoef(x, event_signal)[0,1] for x in signals_detrend.T]

In [ ]:
# Goes to images

amap_raw = unmask(cor_raw, mask_img).get_fdata().squeeze()
amap_detrend = unmask(cor_detrend, mask_img).get_fdata().squeeze()

In [ ]:
# Fancy display => Colorcet is life 

import colorcet as cc

nb_col = 3
nb_lin = 1

fig, ax = plt.subplots(nb_lin, nb_col, figsize = (nb_col*6, nb_lin*5))

ax[0].imshow(np.rot90(amap_raw,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[0].axis('off')

ax[1].imshow(np.rot90(amap_detrend,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[1].axis('off')

ax[2].imshow(np.rot90(amap_detrend-amap_raw,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[2].axis('off')

In [ ]:
# A bit of stats

_ = plt.hist(cor_detrend, 100)

plt.axvline(np.percentile(cor_detrend, 95), c = 'k')

In [ ]:
# Lets get an active zone

mask_active = np.zeros(len(cor_detrend))
mask_active[np.array(cor_detrend) > np.percentile(cor_detrend, 98)] = 1

img_mask_active = unmask(mask_active, mask_img)

In [ ]:
# What does the zone look like

plt.imshow(np.rot90(amap_raw,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)

img_1 = img_mask_active.get_fdata().squeeze()

plt.imshow(np.rot90(img_1,3), cmap = 'Greys', alpha = 0.2)
plt.axis('off')

In [ ]:
# some timeseries

active_signals = apply_mask(img_detrend, img_mask_active)

plt.plot(active_signals.mean(1))

In [ ]:
# Practicals 2.3.1

# Clean image

In [ ]:
# all at once

from nilearn.image import smooth_img, high_variance_confounds

# confounds = high_variance_confounds(img, n_confounds=5, percentile=2.0, detrend=True, mask_img=mask_img)

img_cleaned = clean_img(
    smooth_img(img, fwhm=0.2), 
    mask_img = mask_img, 
    standardize = True,
    detrend = True,
    high_pass = 0.01,
    t_r = 1/fs
)

signals_clean = apply_mask(img_cleaned, mask_img)
cor_clean = [np.corrcoef(x, event_signal)[0,1] for x in signals_clean.T]
amap_clean = unmask(cor_clean, mask_img).get_fdata().squeeze()

In [ ]:
# might be slightly over processed

import colorcet as cc

nb_col = 3
nb_lin = 1

fig, ax = plt.subplots(nb_lin, nb_col, figsize = (nb_col*6, nb_lin*5))

ax[0].imshow(np.rot90(amap_raw,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[0].axis('off')

ax[1].imshow(np.rot90(amap_clean,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[1].axis('off')

ax[2].imshow(np.rot90(amap_clean-amap_raw,3), cmap = cc.cm['CET_D1'], vmin = -0.5, vmax = 0.5)
ax[2].axis('off')

# GLM

Adapted from nilearn tutorials

In [ ]:
# Load data

from nilearn.datasets import fetch_spm_auditory

subject_data = fetch_spm_auditory()
subject_data.func 

In [ ]:
# Display

from nilearn.plotting import plot_anat, plot_img, plot_stat_map

plot_img(subject_data.func[0], colorbar=True, cbar_tick_format="%i")
plot_anat(subject_data.anat, colorbar=True, cbar_tick_format="%i")

In [ ]:
# Builds stim pattern

from nilearn.image import concat_imgs, mean_img
import pandas as pd

# Creates 4D image

fmri_img = concat_imgs(subject_data.func)
mean_img = mean_img(fmri_img)

# extracts stimulation pattern

events = pd.read_table(subject_data["events"])
events

In [ ]:
# creates and fit the model

from nilearn.glm.first_level import FirstLevelModel

fmri_glm = FirstLevelModel(
    t_r=7,
    noise_model="ar1",
    standardize=False,
    hrf_model="spm",
    drift_model="cosine",
    high_pass=0.01,
)

fmri_glm = fmri_glm.fit(fmri_img, events)

In [ ]:
# Checking the model

from nilearn.plotting import plot_design_matrix

design_matrix = fmri_glm.design_matrices_[0]

plot_design_matrix(design_matrix)

plt.show()

In [ ]:
# does the stats

import numpy as np

conditions = {"active": np.zeros(16), "rest": np.zeros(16)}
conditions["active"][0] = 1
conditions["rest"][1] = 1

active_minus_rest = conditions["active"] - conditions["rest"]

eff_map = fmri_glm.compute_contrast(
    active_minus_rest, output_type="effect_size"
)

z_map = fmri_glm.compute_contrast(active_minus_rest, output_type="z_score")

In [ ]:
# plots the stats

plot_stat_map(
    z_map,
    bg_img=mean_img,
    threshold=3.0,
    display_mode="z",
    cut_coords=3,
    black_bg=True,
    title="Active minus Rest (Z>3)",
)
plt.show()

In [ ]:
# Practicals 2.3.2